# Day12
简单 RNN on IMDB 情感分类 (目标准确率 ≥80%)

## 使用 huggingface 进行数据加载与预处理

我们将使用 `huggingface` `datasets` 库来加载 IMDB 数据集，并进行文本的 tokenization、构建词汇表和序列填充。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

In [3]:
# 设置参数
max_features = 10000  # 词汇表大小
maxlen = 200          # 序列最大长度
batch_size = 64       # 批次大小

datasets 库是 Hugging Face 生态系统中一个非常强大、灵活且高效的工具，专门用于加载、处理和分享数据集。它不仅仅是为 NLP 设计的，而是支持各种模态的数据（文本、图像、音频等）。   
datasets 库加载的数据不是简单的 Python 列表或字典，而是一个专门的 Dataset 对象（或 DatasetDict 包含多个子数据集，如 train, validation, test）。
这个 Dataset 对象基于 Apache Arrow (或 Arrow memory mapping)，可以非常高效地处理大型数据集，即使数据集大小超过你的内存容量，它也可以只加载需要的部分到内存。
支持快速访问数据切片、按列操作等。

In [4]:
# 加载数据集
dataset = load_dataset('imdb')

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# 定义预处理函数
def preprocess_function(examples):
    # 对文本进行 tokenization，并进行 padding 和 truncation
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=maxlen)

# 应用预处理函数
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 重命名标签列并移除原始文本列
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# 设置数据格式为 PyTorch tensors
tokenized_datasets.set_format("torch")

print("数据加载和预处理完成。")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

数据加载和预处理完成。


datasets 提供了强大的数据处理 API，你可以用非常简洁的方式对整个数据集或数据集的某个子集进行转换操作。
- map() 方法： 这是最核心的数据处理方法！你可以定义一个函数，然后用 dataset.map(your_function) 将这个函数应用到数据集的每个样本或每个批次上。这使得数据预处理和特征工程变得非常方便。
- filter() 方法： 根据条件过滤数据集的样本。
- remove_columns() / rename_columns()： 移除或重命名数据集的列。
- select()： 选取数据集的某个子集。
- sort()： 对数据集进行排序。

In [5]:
# 创建 DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

print("DataLoader 创建完成。")

DataLoader 创建完成。


In [6]:
# 检查一个批次的数据形状
for batch in train_dataloader:
    print(f"文本批次形状 (input_ids): {batch['input_ids'].shape}")
    print(f"标签批次形状 (labels): {batch['labels'].shape}")
    # Hugging Face tokenizer 还会返回 attention_mask 和 token_type_ids (对于 BERT)
    print(f"Attention mask 批次形状: {batch['attention_mask'].shape}")
    if 'token_type_ids' in batch:
        print(f"Token type ids 批次形状: {batch['token_type_ids'].shape}")
    break

文本批次形状 (input_ids): torch.Size([64, 200])
标签批次形状 (labels): torch.Size([64])
Attention mask 批次形状: torch.Size([64, 200])
Token type ids 批次形状: torch.Size([64, 200])


## 构建简单的 LSTM 模型 

搭建一个包含 Embedding 层、LSTM 层和 Dense (Linear) 层的RNN模型。

In [ ]:
class SimpleLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        # Embedding层，将词ID映射为稠密向量
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # 单层LSTM，batch_first=True保证输入输出的第一个维度是batch
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # 全连接层，将LSTM输出映射为最终分类输出
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        # text: [batch_size, seq_len]
        embedded = self.embedding(text)
        # embedded: [batch_size, seq_len, embedding_dim]
        output, (hidden, cell) = self.lstm(embedded)
        # output: [batch_size, seq_len, hidden_dim]
        # hidden: [1, batch_size, hidden_dim]，取最后一个时间步的隐藏状态
        hidden = hidden.squeeze(0)
        prediction = self.fc(hidden)
        # prediction: [batch_size, output_dim]
        return prediction

In [8]:
# 设置模型参数
# 使用 Hugging Face tokenizer 的词汇表大小
vocab_size = tokenizer.vocab_size
embedding_dim = 128 # 嵌入维度适当提升，兼顾性能和效果
hidden_dim = 128    # LSTM 隐藏层维度适当提升，兼顾性能和效果
output_dim = 1      # 输出维度 (二分类)

# 实例化LSTM模型
model = SimpleLSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)

print("模型构建完成。")
print(model)

模型构建完成。
SimpleLSTMModel(
  (embedding): Embedding(30522, 128)
  (lstm): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


## 训练设置

定义损失函数和优化器。

In [9]:
# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss() # 结合 Sigmoid 和 Binary Cross Entropy Loss，更稳定
optimizer = optim.Adam(model.parameters())

# 将模型和损失函数移动到设备 (如果可用 GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

print(f"使用设备: {device}")
print("损失函数和优化器设置完成。")

使用设备: cpu
损失函数和优化器设置完成。


## 模型训练与评估

定义训练和评估函数，并进行模型训练。

In [ ]:
def binary_accuracy(preds, y):
    """
    计算每个批次的准确率
    """
    # 对预测结果进行四舍五入到最接近的整数
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() # 转换为浮点数进行除法
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() # 设置模型为训练模式
    
    for batch in iterator:
        # 从 Hugging Face DataLoader 的批次中提取 input_ids 和 labels
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad() # 清零梯度
        
        predictions = model(input_ids).squeeze(1) # 移除维度为 1 的维度
        #保证criterion接受输入shape一致
        loss = criterion(predictions, labels.float()) # 标签需要是浮点数
        
        acc = binary_accuracy(predictions, labels)
        loss.backward() # 反向传播
        optimizer.step() # 更新权重
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval() # 设置模型为评估模式
    
    with torch.no_grad(): # 在评估阶段不计算梯度
    
        for batch in iterator:
            # 从 Hugging Face DataLoader 的批次中提取 input_ids 和 labels
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            predictions = model(input_ids).squeeze(1)
            loss = criterion(predictions, labels.float())
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
train_losses, valid_losses, train_accs, valid_accs = [], [], [], []

epochs = 10 # 不要继续训练了，过拟合严重
best_valid_loss = float('inf')
best_epoch = 0
print("start training...")
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_dataloader, criterion)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss,
        }
        torch.save(checkpoint, f'../model/SimpleLSTM/checkpoint_{epoch+1}.pth')
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Valid. Loss: {valid_loss:.3f} |  Valid. Acc: {valid_acc*100:.2f}%')

start training...
Epoch: 01
	Train Loss: 0.644 | Train Acc: 62.61%
	 Valid. Loss: 0.560 |  Valid. Acc: 73.64%
Epoch: 02
	Train Loss: 0.433 | Train Acc: 80.95%
	 Valid. Loss: 0.417 |  Valid. Acc: 80.87%
Epoch: 03
	Train Loss: 0.291 | Train Acc: 88.50%
	 Valid. Loss: 0.373 |  Valid. Acc: 83.77%
Epoch: 04
	Train Loss: 0.213 | Train Acc: 92.15%
	 Valid. Loss: 0.407 |  Valid. Acc: 84.17%
Epoch: 05
	Train Loss: 0.148 | Train Acc: 95.11%
	 Valid. Loss: 0.432 |  Valid. Acc: 84.06%
Epoch: 06
	Train Loss: 0.103 | Train Acc: 96.88%
	 Valid. Loss: 0.540 |  Valid. Acc: 82.78%
Epoch: 07
	Train Loss: 0.068 | Train Acc: 98.15%
	 Valid. Loss: 0.540 |  Valid. Acc: 83.11%
Epoch: 08
	Train Loss: 0.050 | Train Acc: 98.66%
	 Valid. Loss: 0.696 |  Valid. Acc: 83.07%
Epoch: 09
	Train Loss: 0.040 | Train Acc: 98.95%
	 Valid. Loss: 0.696 |  Valid. Acc: 83.42%
Epoch: 10
	Train Loss: 0.030 | Train Acc: 99.28%
	 Valid. Loss: 0.763 |  Valid. Acc: 83.09%


模型在epoch3之后开始过拟合。(valid loss上升)   
可以增加dropout层，但最好在多层RNN中使用。   
不对该单层LSTM做优化。   

In [19]:
# 加载最优模型并评估
checkpoint = torch.load(f'../model/SimpleLSTM/checkpoint_{best_epoch+1}.pth')

model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

test_loss, test_acc = evaluate(model, test_dataloader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.373 | Test Acc: 83.77%


In [20]:
# 单条文本预测函数
def predict_sentiment(model, tokenizer, sentence, device, maxlen=200):
    model.eval()
    tokens = tokenizer(sentence, padding='max_length', truncation=True, max_length=maxlen, return_tensors='pt')
    input_ids = tokens['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids)
        prob = torch.sigmoid(output.squeeze(1)).item()
    return prob

# 示例：预测一条影评
sample_text = "This movie was absolutely fantastic! I loved it."
prob = predict_sentiment(model, tokenizer, sample_text, device)
print(f'正面情感概率: {prob:.4f}')
print('预测标签:', 'positive' if prob > 0.5 else 'negative')

正面情感概率: 0.9527
预测标签: positive
